In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import keras
import tensorflow as tf
from nltk.corpus import stopwords
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from nltk.stem import PorterStemmer
import string
data = pd.read_json("./Sarcasm_Headlines_Dataset_v2.json",lines=True)

In [2]:
data

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...
...,...,...,...
28614,1,jews to celebrate rosh hashasha or something,https://www.theonion.com/jews-to-celebrate-ros...
28615,1,internal affairs investigator disappointed con...,https://local.theonion.com/internal-affairs-in...
28616,0,the most beautiful acceptance speech this week...,https://www.huffingtonpost.com/entry/andrew-ah...
28617,1,mars probe destroyed by orbiting spielberg-gat...,https://www.theonion.com/mars-probe-destroyed-...


In [3]:
import nltk
nltk.download('stopwords')

stwrds = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
# remove duplicate headlines
data=data.drop(data[data['headline'].duplicated()].index,axis=0)
sarc_cnt = len(data.query('is_sarcastic==1'))
non_sarc_cnt = len(data.query('is_sarcastic==0'))

# Summary of sarcastic lines
print(f'There are {sarc_cnt} sarcastic headlines and {non_sarc_cnt} non-sarcastic headlines')
# import stopwords from nltk
stwrds = set(stopwords.words('english'))
ps = PorterStemmer()
# method to clean a given headline by lowercasing the string, removing spaces, and removing stopwords
def clean_headlines(headline):
    headline = headline.lower()
    headline_split = headline.split()
    cleaned_headline = []
    for word in headline_split:
        if word not in stwrds and word not in string.punctuation:
            cleaned_headline.append(ps.stem(word))
    cleaned_line = " ".join(cleaned_headline)
    return cleaned_line
data['cleaned'] = data['headline'].apply(clean_headlines)

There are 13552 sarcastic headlines and 14951 non-sarcastic headlines


In [5]:
labels=data.is_sarcastic 
features=data.cleaned

In [6]:
feature_train,feature_test, labels_train,labels_test = train_test_split(features,labels ,test_size =0.2)

In [7]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(feature_train)

vocab_size = len(tokenizer.word_index)
print(vocab_size)

train_sequences = tokenizer.texts_to_sequences(feature_train)
test_sequences = tokenizer.texts_to_sequences(feature_test)

21855


In [8]:
maxlen=max([len(i) for i in train_sequences])

In [9]:
train_padded = pad_sequences(train_sequences, maxlen=maxlen,  padding='post')
test_padded = pad_sequences(test_sequences, maxlen=maxlen,  padding='post')

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
# from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [11]:
clf = DecisionTreeClassifier(criterion="entropy", max_depth=6)
clf = clf.fit(train_padded,labels_train)
# predictions_binary = clf.predict(test_padded)


In [12]:
print(classification_report(labels_test, clf.predict(test_padded)))

              precision    recall  f1-score   support

           0       0.59      0.84      0.70      2978
           1       0.68      0.37      0.48      2723

    accuracy                           0.62      5701
   macro avg       0.64      0.61      0.59      5701
weighted avg       0.64      0.62      0.59      5701



In [13]:
from tqdm import tqdm

In [14]:
for i in tqdm (range (2,20,2)):
    if i in[0,1,10]: continue
    clf = DecisionTreeClassifier(criterion="entropy", max_depth=i)
    clf.fit(train_padded,labels_train)
    print("Max depth = ", i)
    print(classification_report(labels_test, clf.predict(test_padded)))

 11%|█         | 1/9 [00:00<00:01,  6.61it/s]

Max depth =  2
              precision    recall  f1-score   support

           0       0.59      0.84      0.69      2978
           1       0.67      0.35      0.46      2723

    accuracy                           0.61      5701
   macro avg       0.63      0.60      0.57      5701
weighted avg       0.62      0.61      0.58      5701



 22%|██▏       | 2/9 [00:00<00:01,  5.34it/s]

Max depth =  4
              precision    recall  f1-score   support

           0       0.59      0.84      0.69      2978
           1       0.67      0.35      0.46      2723

    accuracy                           0.61      5701
   macro avg       0.63      0.60      0.57      5701
weighted avg       0.62      0.61      0.58      5701



 33%|███▎      | 3/9 [00:00<00:01,  4.18it/s]

...

Max depth =  14
              precision    recall  f1-score   support

           0       0.60      0.72      0.66      2978
           1       0.61      0.47      0.53      2723

    accuracy                           0.60      5701
   macro avg       0.60      0.60      0.59      5701
weighted avg       0.60      0.60      0.60      5701



 89%|████████▉ | 8/9 [00:02<00:00,  2.56it/s]

Max depth =  16
              precision    recall  f1-score   support

           0       0.60      0.71      0.65      2978
           1       0.60      0.48      0.53      2723

    accuracy                           0.60      5701
   macro avg       0.60      0.59      0.59      5701
weighted avg       0.60      0.60      0.59      5701



100%|██████████| 9/9 [00:03<00:00,  2.89it/s]

Max depth =  18
              precision    recall  f1-score   support

           0       0.60      0.70      0.65      2978
           1       0.60      0.48      0.53      2723

    accuracy                           0.60      5701
   macro avg       0.60      0.59      0.59      5701
weighted avg       0.60      0.60      0.59      5701

